In [13]:
import os
import requests
import json
import time
from datetime import datetime

In [ ]:
# Set up the Azure OpenAI API endpoint and deployment details
endpoint = "https://YOUR_SERVICE_NAME.openai.azure.com/"  
deployment = "sora"
api_key = "YOUR_KEY"
api_version = "2025-02-15-preview"

In [ ]:
headers = {
  'Api-Key': api_key,
  'Content-Type': 'application/json',
}

In [ ]:
# Set up prompt
#prompt = "A seamless 10-second loop of a white sandy beach and emerald green sea under a blue sky in morning light. The scene is viewed from a static camera with a natural human-eye perspective (around 50mm focal length), avoiding fisheye or wide-angle distortion. Gentle waves roll in and out in a perfectly cyclical motion. Ensure the first and last frames are visually identical for a perfect loop. Maintain consistent lighting and shadows throughout"
prompt = "In a bright afternoon office, several Japanese people dressed in casual are relaxing while enjoying coffee and sweets. The camera is static, using a natural human-eye perspective (around 50mm focal length), avoiding fisheye or wide-angle distortion. Their movements and expressions are calm and natural. Ensure the first and last frames match visually for a seamless loop. Maintain consistent lighting and shadows throughout. The video should be a smooth, 5-second seamless loop."

In [15]:
body = {
  "prompt": prompt,
  "n_variants": "1",
  "n_seconds": "10",
  "height": "480",
  "width": "854",
  "model": deployment,
}

In [ ]:
# Execute the request to generate the video
constructed_url = f"{endpoint}openai/v1/video/generations/jobs?api-version=preview"
response = requests.post(constructed_url, headers=headers, json=body)

if not response.ok:
    print("Video generation failed.")
    print(json.dumps(response.json(), sort_keys=True, indent=4, separators=(',', ': ')))
else:
    print(json.dumps(response.json(), sort_keys=True, indent=4, separators=(',', ': ')))
    response = response.json()
    job_id = response.get("id")
    status = response.get("status")
    status_url = f"{endpoint}openai/v1/video/generations/jobs/{job_id}?api-version=preview"

    print(f"Polling job status for ID: {job_id}")
    while status not in ["succeeded", "failed"]:
        time.sleep(5)
        response = requests.get(status_url, headers=headers).json()
        status = response.get("status")
        print(f"Status: {status}")

    if status == "succeeded":
        generations = response.get("generations", [])
        if generations:
            print(f"Video generation succeeded.")

            generation_id = generations[0].get("id")
            video_url = f'{endpoint}openai/v1/video/generations/{generation_id}/content/video?api-version=preview'
            video_response = requests.get(video_url, headers=headers)
            if video_response.ok:
                output_filename = "./videos/generated/" + datetime.now().strftime("%Y%m%d_%H%M%S") + ".mp4"
                with open(output_filename, "wb") as file:
                    file.write(video_response.content)
                print(f'Generated video saved as "{output_filename}"')
        else:
            print("Status is succeeded, but no generations were returned.")
    elif status == "failed":
        print("Video generation failed.")
        print(json.dumps(response, sort_keys=True, indent=4, separators=(',', ': ')))

{
    "created_at": 1748926604,
    "expires_at": null,
    "failure_reason": null,
    "finished_at": null,
    "generations": [],
    "height": 480,
    "id": "task_01jwt2c9m2fg2a5d7qf4wpn5ay",
    "model": "sora",
    "n_seconds": 10,
    "n_variants": 1,
    "object": "video.generation.job",
    "prompt": ":In a bright afternoon office, several Japanese people dressed in casual are relaxing while enjoying coffee and sweets. The camera is static, using a natural human-eye perspective (around 50mm focal length), avoiding fisheye or wide-angle distortion. Their movements and expressions are calm and natural. Ensure the first and last frames match visually for a seamless loop. Maintain consistent lighting and shadows throughout. The video should be a smooth, 5-second seamless loop",
    "status": "queued",
    "width": 854
}
Polling job status for ID: task_01jwt2c9m2fg2a5d7qf4wpn5ay
Status: queued
Status: queued
Status: queued
Status: queued
Status: running
Status: running
Status: runn

In [ ]:
# Execute the request to generate the video
creation_url = f"{endpoint}openai/deployments/{deployment}/video/generations/generate?api-version={api_version}"
response = requests.post(constructed_url, headers=headers, json=body)

if not response.ok:
    print("Video generation failed.")
    print(json.dumps(response.json(), sort_keys=True, indent=4, separators=(',', ': ')))
else:
    print(json.dumps(response.json(), sort_keys=True, indent=4, separators=(',', ': ')))
    response = response.json()
    job_id = response.get("id")
    status = response.get("status")
    status_url = f"{endpoint}openai/v1/video/generations/jobs/{job_id}?api-version=preview"

    print(f"Polling job status for ID: {job_id}")
    while status not in ["succeeded", "failed"]:
        time.sleep(5)
        response = requests.get(status_url, headers=headers).json()
        status = response.get("status")
        print(f"Status: {status}")

    if status == "succeeded":
        generations = response.get("generations", [])
        if generations:
            print(f"Video generation succeeded.")

            generation_id = generations[0].get("id")
            video_url = f'{endpoint}openai/v1/video/generations/{generation_id}/content/video?api-version=preview'
            video_response = requests.get(video_url, headers=headers)
            if video_response.ok:
                output_filename = "./videos/generated/" + datetime.now().strftime("%Y%m%d_%H%M%S") + ".mp4"
                with open(output_filename, "wb") as file:
                    file.write(video_response.content)
                print(f'Generated video saved as "{output_filename}"')
        else:
            print("Status is succeeded, but no generations were returned.")
    elif status == "failed":
        print("Video generation failed.")
        print(json.dumps(response, sort_keys=True, indent=4, separators=(',', ': ')))